In [ ]:
%load_ext autoreload
%autoreload 2

# !git clone https://github.com/Flagro/ExcelTableCNN.git

!apt-get install -y libreoffice --fix-missing

!pip install py7zr
!pip install tqdm
!pip install torchvision
!pip install torch

In [ ]:
%cd /kaggle/working/ExcelTableCNN
!git pull origin main

In [ ]:
from ExcelTableCNN.excel_table_cnn.train_test_helpers.train_test_composer import get_train_test

train_df, test_df = get_train_test(train_size=12, testing_size=4,
                                   data_folder_path="/kaggle/working/ExcelTableCNN/data")

In [ ]:
from ExcelTableCNN.excel_table_cnn.dl_classification.tensors import SpreadsheetDataset

train_dataset = SpreadsheetDataset(train_df)
test_dataset = SpreadsheetDataset(test_df)

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

batch_size = 1  # For different-sized inputs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
import torch
from ExcelTableCNN.excel_table_cnn.dl_classification.train_eval import get_model

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model()

In [ ]:
import torch.optim as optim
from ExcelTableCNN.excel_table_cnn.dl_classification.train_eval import train_model

optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
train_model(model, train_loader, optimizer, 10, device)

In [ ]:
from ExcelTableCNN.excel_table_cnn.dl_classification.train_eval import evaluate_model

evaluate_model(model, test_loader, device)